# PyBADS Example 3: Noisy objective function

In this example, we will show how to run PyBADS on a noisy target.

This notebook is Part 3 of a series of notebooks in which we present various example usages for BADS with the PyBADS package.

In [1]:
import numpy as np
from pybads.bads.bads import BADS

## 0. Noisy optimization

PyBADS is able to optimize also *noisy* objective functions. A noisy (or stochastic) objective function is an objective that will return different results if evaluated twice at the same point $\mathbf{x}$. Conversely, a non-noisy objective function is known as noiseless or deterministic. For example, noisy objectives are common in model fitting when the model is evaluated through simulation (e.g., via sampling aka Monte Carlo methods).

For a noisy objective, PyBADS attempts to minimize the *expected value* of $f(\mathbf{x})$,
$$
\mathbf{x}^\star = \arg\min_{\mathbf{x} \in \mathcal{X} \subseteq \mathbb{R}^D} \mathbb{E}\left[f(\mathbf{x})\right].
$$

## 1. Problem definition

We optimize [Rosenbrock's banana function](https://en.wikipedia.org/wiki/Rosenbrock_function) in 2D as in the [previous example](./pybads_example_1_basic_usage.ipynb), but here we force the input to stay within a circle with unit radius.

Since we know the optimization region, we set tight box bounds `lb` and `ub` around it to further help the search.

Note that `nonbndcons` is a function that defines constraints *violation*. Specifically, it takes as input a $M$-by-$D$ array $\mathbf{x}_1, \ldots, \mathbf{x}_M$, where each $\mathbf{x} \in \mathbb{R}^D$, and outputs a $M$-by-1 `bool` array. The $m$-th value of the output array is `True` if $\mathbf{x}_m$ violates the constraint, `False` otherwise.

In [2]:
def noisy_quadratic_fcn(x):
    """Simple quadratic function with added noise."""
    x_2d = np.atleast_2d(x)
    return np.sum(100 * (x_2d[:, 0:-1]**2 - x_2d[:, 1:])**2 + (x_2d[:, 0:-1]-1)**2, axis=1)

x0 = np.array([[-3, -3]]);      # Starting point
lb = np.array([[-5, -5]])       # Lower bounds
ub = np.array([[5, 5]])         # Upper bounds
plb = np.array([[-2, -2]])      # Plausible lower bounds
pub = np.array([[2, 2]])        # Plausible upper bounds


def circle_constr(x):
    """Return constraints violation outside the unit circle."""
    x_2d = np.atleast_2d(x)
    # Note that nonboxcons assumes the function takes a 2D input 
    return np.sum(x_2d**2, axis=1) > 1

## 2. Run the optimization

We initialize `bads` with the non-box constraints defined by `nonboxcons`. Note that we also still specify standard box constraints `lb` and `ub`, as this will help the search.

Incidentally, BADS will complain because the plausible bounds are not specified explicitly. In the absence of plausible bounds, BADS will create them based on the lower/upper bounds instead. Generally, you should specify the plausible bounds.

In [3]:
bads = BADS(rosenbrocks_fcn, x0, lb, ub, None, None, nonbondcons=circle_constr)
x_min, fval = bads.optimize()

bads:TooCloseBounds: For each variable, hard and plausible bounds should not be too close. Moving plausible bounds.
Variables (index) defined with periodic boundaries: []
Beginning optimization of a DETERMINISTIC objective function

 Iteration f-count     f(x)     MeshScale     Method     Actions
     0         3       1.000000      1.000000            Uncertainty test
     0         7       1.000000      1.000000     Initial mesh       Initial points


c:\users\luigi\documents\github\gpyreg\gpyreg\covariance_functions.py:376: RuntimeWarning: invalid value encountered in add
  plausible_lower_bounds[i_nan] + plausible_upper_bounds[i_nan]


     0        11       1.000000      0.500000     Refine grid       Train
     1        15       0.652949      0.500000     Incremental search (('ES-wcm', 1))       
     1        19       0.652949      0.250000     Refine grid       Train
     2        20       0.280228      0.250000     Successful search (('ES-wcm', 1))       
     2        21       0.008201      0.250000     Successful search (('ES-wcm', 1))       
     2        22       0.000000      0.250000     Incremental search (('ES-ell', 1))       
     2        29       0.000000      0.125000     Refine grid       
     3        35       0.000000      0.062500     Refine grid       
     4        37       0.000000      0.062500     Incremental search (('ES-wcm', 1))       


c:\users\luigi\documents\github\pybads\pybads\bads\bads.py:1760: RuntimeWarning: divide by zero encountered in true_divide
  zscore = zscore / gp_ys


bads:_robust_gp_fit_: posterior GP update failed. Singular matrix for L Cholesky decomposition
bads:_robust_gp_fit_: posterior GP update failed. Singular matrix for L Cholesky decomposition
     4        41       0.000000      0.031250     Refine grid       Train
bads: The optimization is stalling, decreasing further the mesh size
     5        47       0.000000      0.007812     Refine grid       Train
bads:_robust_gp_fit_: posterior GP update failed. Singular matrix for L Cholesky decomposition
bads:_robust_gp_fit_: posterior GP update failed. Singular matrix for L Cholesky decomposition
bads: The optimization is stalling, decreasing further the mesh size
     6        53       0.000000      0.001953     Refine grid       Train
Optimization terminated: change in the function value less than options.TolFun.
Function value at minimum: 5.839503487943903e-08



## 3. Results and conclusions

In [4]:
print(f"BADS minimum at: x = {x_min.flatten()}, fval = {fval:.4g}")
print(f"total f-count: {bads.function_logger.func_count-1}, time: {round(bads.optim_state['total_time'], 2)} s")

BADS minimum at: x = [0.99998729 0.99999871], fval = 5.84e-08
total f-count: 53, time: 5.25 s


The true global minimum of the Rosenbrock function under these constraints is at $\textbf{x}^\star_\text{min} = [0.786,0.618]$, where $f^\star_\text{min} = 0.046$.

*Note*: While in theory `nonboxcons` can receive any arbitrary constraints, in practice PyBADS will likely work well only within relatively simple domains (e.g., simple convex regions), as the current version of (Py)BADS uses a simple heuristic to reject samples outside the admissible region.

## Example 2: Full code

See [here](./src/pybads_example_2_nonbox_constraints.py) for a Python file with the code used in this example, with no extra fluff.

In [ ]:
import numpy as np
from pybads.bads.bads import BADS
from pybads.bads.bads_dump import BADSDump
from pybads.function_examples import quadratic_unknown_noisy_fcn, quadratic_noisy_fcn, extra_noisy_quadratic_fcn

x0 = np.array([[-3, -3]]);        # Starting point
lb = np.array([[-5, -5]])     # Lower bounds
ub = np.array([[5, 5]])       # Upper bounds
plb = np.array([[-2, -2]])      # Plausible lower bounds
pub = np.array([[2, 2]])        # Plausible upper bounds

title = 'Noise objective function'
print("\n *** Example 3: " + title)
print("\t We test BADS on a noisy quadratic function with unit Gaussian noise.")
# Remarks:  For testing the heteroskedastic/user noise use quadratic_noisy_fcn as input to BADS
#           and set in basic_bads_option.ini to True uncertaintyhandling and specifytargetnoise options.
bads = BADS(quadratic_unknown_noisy_fcn, x0, lb, ub, plb, pub)

x_min, fval = bads.optimize()
print(f"BADS minimum at: \n\n\t x = {x_min.flatten()} \n\t fval= {fval} \n\t \
    total time: {round(bads.optim_state['total_time'], 2)} s \n overhead: {round(bads.optim_state['overhead'], 2)}")
x_global_min = np.array([0., 0.])
print(f"The true, noiseless minimum is at x = {np.sum(x_min**2)} \n")
print(f"The true global minimum is at x = [0, 0], where fval = 0\n")

bads_dump = BADSDump("stobads_noise")
bads_dump.to_JSON(bads.x, bads.u, bads.fval, bads.fsd, bads.iteration_history,
            x_global_min)

extra_noise = True
if extra_noise:
    title = 'Extra Noise objective function'
    print("\n *** Example 4: " + title)
    print("\t We test BADS on a particularly noisy function.")
    bads = BADS(extra_noisy_quadratic_fcn, x0, lb, ub, plb, pub)
    x_min, fval = bads.optimize()
    print(f"BADS minimum at: \n\n\t x = {x_min.flatten()} \n\t fval= {fval} \n\t \
    total time: {round(bads.optim_state['total_time'], 2)} s \n overhead: {round(bads.optim_state['overhead'], 2)}")
    print(f"The true global minimum is at x = [1, 1], where fval = 0\n")